# Environmental Impact Analysis using Satellite Data

### Step 1: Define the Objective and Scope of the Analysis
#### a. Identify the Specific Environmental Issue
For example, let’s choose to analyze deforestation.

#### b. Geographic Scope
Define the specific geographic area of interest, e.g., Amazon Rainforest.

#### c. Temporal Scope
Decide on the time frame for your analysis, e.g., January 2019 to December 2023.

#### d. Data Requirements
We'll use the Harmonized Sentinel-2 MSI: MultiSpectral Instrument, Level-1C data, which provides high-resolution optical images.

#### e. Expected Outcomes
The analysis aims to:
        Identify areas of deforestation over the specified period.
        Quantify the rate of deforestation.
        Provide visualizations and statistical data to support findings.

In [1]:
import ee

# Authenticate the Earth Engine API
ee.Authenticate()

# Initialize the Earth Engine API
ee.Initialize()


In [2]:
import geemap  # Interactive mapping with GEE
import matplotlib.pyplot as plt  # Plotting library
import pandas as pd  # Data manipulation and analysis
import numpy as np  # Numerical operations
import seaborn as sns  # Statistical data visualization
from shapely.geometry import Polygon  # Geometric objects manipulation

In [3]:
# Define the area of interest
aoi = ee.Geometry.Polygon([
    [[-63.0, -10.0], [-63.0, -5.0], [-58.0, -5.0], [-58.0, -10.0], [-63.0, -10.0]]
])

# Define the time frame for the analysis
start_date = '2019-01-01'
end_date = '2023-12-31'

# Load the Sentinel-2 dataset
sentinel2 = ee.ImageCollection("COPERNICUS/S2_HARMONIZED") \
    .filterBounds(aoi) \
    .filterDate(start_date, end_date) \
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20))

### Step 2: Preprocess the Satellite Data 
In this step, we will preprocess the satellite data to prepare it for analysis. Preprocessing includes steps like masking clouds, applying necessary image transformations, and generating a composite image.

In [4]:
# Cloud Masking Function
def mask_clouds(image):
    qa = image.select('QA60')
    cloud_bit_mask = 1 << 10
    cirrus_bit_mask = 1 << 11
    mask = qa.bitwiseAnd(cloud_bit_mask).eq(0).And(
        qa.bitwiseAnd(cirrus_bit_mask).eq(0))
    return image.updateMask(mask)

# Apply the cloud masking function
sentinel2_cloud_free = sentinel2.map(mask_clouds)

In [5]:
# Calculate NDVI (Normalized Difference Vegetation Index)
def add_ndvi(image):
    ndvi = image.normalizedDifference(['B8', 'B4']).rename('NDVI')
    return image.addBands(ndvi)

# Add NDVI band to each image
sentinel2_ndvi = sentinel2_cloud_free.map(add_ndvi)

In [6]:
#Create Annual NDVI Composite Images
def annual_composite(year):
    start = ee.Date.fromYMD(year, 1, 1)
    end = ee.Date.fromYMD(year, 12, 31)
    composite = sentinel2_ndvi.filterDate(start, end).median()
    return composite.set('year', year).addBands(ee.Image.constant(year).rename('year'))

years = ee.List.sequence(2019, 2023)
annual_ndvi_composites = ee.ImageCollection(years.map(annual_composite))


In [7]:
# Initialize a map
Map = geemap.Map()

# Define visualization parameters for NDVI
ndvi_params = {
    'min': 0,
    'max': 1,
    'palette': ['white', 'green']
}

# Add the 2019 NDVI composite to the map
ndvi_2019 = annual_ndvi_composites.filter(ee.Filter.eq('year', 2019)).first()
Map.centerObject(aoi, 8)
Map.addLayer(ndvi_2019.select('NDVI'), ndvi_params, 'NDVI 2019')
Map.addLayerControl()  # Add layer control to the map
Map


Map(center=[-7.502247125720452, -60.49999999999998], controls=(WidgetControl(options=['position', 'transparent…

### Step 3: Analyze Changes Over Time
In this step, we will analyze the changes in NDVI over the specified time period to identify areas of deforestation. This involves calculating the difference in NDVI between the start and end of the analysis period and identifying areas with significant decreases in NDVI.

In [8]:
# Get the NDVI composites for 2019 and 2023
ndvi_2019 = annual_ndvi_composites.filter(ee.Filter.eq('year', 2019)).first().select('NDVI')
ndvi_2023 = annual_ndvi_composites.filter(ee.Filter.eq('year', 2023)).first().select('NDVI')

# Calculate the NDVI change
ndvi_change = ndvi_2023.subtract(ndvi_2019).rename('NDVI_Change')

# Define visualization parameters for NDVI change
ndvi_change_params = {
    'min': -0.5,
    'max': 0.5,
    'palette': ['red', 'white', 'green']
}

# Add the NDVI change layer to the map
Map.addLayer(ndvi_change, ndvi_change_params, 'NDVI Change 2019-2023')
Map

Map(center=[-7.502247125720452, -60.49999999999998], controls=(WidgetControl(options=['position', 'transparent…

In [9]:
# Define a threshold for significant NDVI decrease
threshold = -0.2

# Create a binary image where significant deforestation areas are marked
deforestation_areas = ndvi_change.lt(threshold).selfMask()

# Define visualization parameters for deforestation areas
deforestation_params = {
    'palette': ['red']
}

# Add the deforestation areas layer to the map
Map.addLayer(deforestation_areas, deforestation_params, 'Significant Deforestation')
Map

Map(center=[-7.502247125720452, -60.49999999999998], controls=(WidgetControl(options=['position', 'transparent…

In [10]:
# Define a threshold for significant NDVI decrease
threshold = -0.2

# Create a binary image where significant deforestation areas are marked
deforestation_areas = ndvi_change.lt(threshold).selfMask()

# Define visualization parameters for deforestation areas
deforestation_params = {
    'palette': ['red']
}

# Add the deforestation areas layer to the map
Map.addLayer(deforestation_areas, deforestation_params, 'Significant Deforestation')
Map

Map(center=[-7.502247125720452, -60.49999999999998], controls=(WidgetControl(options=['position', 'transparent…

### Step 4: Quantify Deforestation

In [11]:
# Convert deforestation areas to vectors within the specified AOI with larger scale and best effort
deforestation_vectors = deforestation_areas.reduceToVectors(
    geometry=aoi,  # Specify the AOI as the geometry parameter
    geometryType='polygon',
    reducer=ee.Reducer.countEvery(),
    scale=2000,  # Use a larger scale to reduce computational load
    maxPixels=1e9,  # Increase the maxPixels limit
    bestEffort=True  # Use best effort to reduce computation load
)

# Function to calculate the area of each polygon with an error margin
def calculate_area(feature):
    area = feature.geometry().area(maxError=1).divide(1e4)  # Convert to hectares with a non-zero error margin
    return feature.set({'area': area})

# Calculate the area for each polygon
deforestation_areas_with_area = deforestation_vectors.map(calculate_area)

# Print the number of deforestation vectors
print(f"Number of deforestation vectors: {deforestation_areas_with_area.size().getInfo()}")

# Print the first few features to inspect
deforestation_areas_list = deforestation_areas_with_area.limit(10).getInfo()
print(f"First few deforestation areas with calculated area: {deforestation_areas_list}")

# Get the total deforested area
deforestation_stats = deforestation_areas_with_area.reduceColumns(
    reducer=ee.Reducer.sum(),
    selectors=['area']
)

# Print the deforestation stats
deforestation_stats_info = deforestation_stats.getInfo()
print(f"Deforestation stats: {deforestation_stats_info}")

# Get the total deforested area
if 'sum' in deforestation_stats_info:
    total_deforested_area = deforestation_stats_info['sum']
    print(f"Total Deforested Area from 2019 to 2023: {total_deforested_area:.2f} hectares")
else:
    print("No deforested area detected.")


Number of deforestation vectors: 316
First few deforestation areas with calculated area: {'type': 'FeatureCollection', 'columns': {'area': 'Number', 'count': 'Long<0, 4294967295>', 'label': 'Byte<0, 1>', 'system:index': 'String'}, 'features': [{'type': 'Feature', 'geometry': {'geodesic': False, 'type': 'Polygon', 'coordinates': [[[-58.03116735412109, -9.270613732113462], [-58.03116735412109, -9.288580037795853], [-58.0132010484387, -9.288580037795853], [-58.0132010484387, -9.306546343478242], [-57.99523474275631, -9.306546343478242], [-57.99523474275631, -9.288580037795853], [-58.0132010484387, -9.288580037795853], [-58.0132010484387, -9.270613732113462], [-58.03116735412109, -9.270613732113462]]]}, 'id': '-3230-517', 'properties': {'area': 787.7461923776476, 'count': 2, 'label': 1}}, {'type': 'Feature', 'geometry': {'geodesic': False, 'type': 'Polygon', 'coordinates': [[[-58.085066271168266, -9.98926595940908], [-58.06709996548587, -9.98926595940908], [-58.06709996548587, -9.971299653

In [12]:
# Define visualization parameters for deforested areas with area information
deforestation_vector_params = {
    'color': 'red'
}

# Add the deforested areas with area information to the map
Map.addLayer(deforestation_areas_with_area, deforestation_vector_params, 'Deforested Areas with Area Info')
Map


Map(center=[-7.502247125720452, -60.49999999999998], controls=(WidgetControl(options=['position', 'transparent…

In [13]:
import ee
import time

# Authenticate the Earth Engine session with Google Drive scope
ee.Authenticate(scopes=['https://www.googleapis.com/auth/earthengine', 'https://www.googleapis.com/auth/drive'])

# Initialize Earth Engine with the authenticated credentials
ee.Initialize()

# Assuming deforestation_areas_with_area is defined somewhere in your code
# Remove the geometry from the features
deforestation_areas_attributes = deforestation_areas_with_area.map(lambda f: f.setGeometry(None))

# Define the export task
export_task = ee.batch.Export.table.toDrive(
    collection=deforestation_areas_attributes,
    description='Deforested_Areas_2019_2023',
    fileFormat='CSV',
    folder='EarthEngine'  # Optional: specify a folder in your Google Drive
)

# Start the export task
export_task.start()

# Monitor the task status
while export_task.status()['state'] in ['READY', 'RUNNING']:
    print('Exporting... Current status: {}'.format(export_task.status()['state']))
    time.sleep(30)

# Print final status
print('Export task completed with status: {}'.format(export_task.status()['state']))


EEException: Request had insufficient authentication scopes.